# Value-at-Risk for Stocks: Historical Simulation

### Lecture Notes by Jakov Ivan S. Dumbrique (jdumbrique@ateneo.edu)

MA 195L.2: Introduction to Financial Mathematics II \
Second Semester, S.Y. 2020-2021 \
Ateneo de Manila University

In [1]:
import numpy as np # Numerical Computing
import pandas as pd # Data wrangling
import matplotlib.pyplot as plt # Plotting
from scipy.stats import norm # statistical analysis
from statistics import NormalDist # statistical analysis

%matplotlib inline


## Question on Single-Asset Portfolio:
Today is February 23, 2018. You are a portfolio risk manager who is assigned to analyze the market risk of a portfolio of 700 PLDT (TEL) shares. Determine the portfolio's one-day 99% VaR using historical simulation approach.
 


In [4]:
def get_return(df, d):
    """
    df is the original df
    
    appends returns series to df
    """
    df["previous"] = df["close"].shift(-d)
    df["return"] = np.log(df["close"]/df["previous"])
    return df 

In [14]:
def get_change_in_value_df(df, N):
    """
    df is the output of get_returns_df 
    
    appends change in portfolio value series to df
    """
    S0 = df.loc[0, "close"]
    df["change_in_value"] = N * S0 * df["return"]
    return df

In [37]:
def get_kth_percentile_discrete(df, d, p):
    """
    this returns the (1-p/100)th percentile of the ordered array of historical changes in portfolio values
    """
    M = len(df["change_in_value"])-d
    alpha = p/100
    k = int(np.floor((1-alpha)*M))
    var = abs(df["change_in_value"].nsmallest(k).values[-1])
    
    return var

In [38]:
def d_day_p_percent_VaR_single_stock_historical(
    df, N, d, p
):
    """Returns the d-day p% VaR of a single stock using Historical Simulation Approach.
    
    Parameters
    ----------
    df : pandas.DataFrame
        has two columns: (1) dt [str] and (2) closing price [float]
        assumes the dates are arranged from newest to oldest, and the date today is the date on the first row  
    N : int
        number of shares for the sole stock
    d : int
        the value to be used in calculating the d-day VaR (e.g. 1-day, 5-day)
    p : float
        the value to be used in calculting the p% VaR (e.g. 99, 95, 97.5)
    
    Returns
    -------
    float (2 decimal places)
        d-day p% VaR of a single stock using Historical Simulation Approach
    """
    
    # step 1: generate your historical returns
    df = get_return(df, d)
    # Step 2: get your historical changes in portfolio values
    df = get_change_in_value_df(df, N)
    # Step 3: get the (1-p/100)th percentile of the ordered array of historical changes in portfolio values
    var = get_1minuskth_percentile_discrete(df, d, p)
    
    return round(var, 2)

In [8]:
TEL_df = pd.read_csv("../data/TEL_2018.csv")

In [39]:
d_day_p_percent_VaR_single_stock_historical(df=TEL_df, N=700, d=1, p=99)

60730.66

## Scratchwork

In [9]:
TEL_df

,dt,close
0,2/23/18,1488.74
1,2/22/18,1510.86
2,2/21/18,1513.72
3,2/20/18,1536.65
4,2/16/18,1476.37
...,...,...
243,3/2/17,1430.29
244,3/1/17,1408.94
245,2/28/17,1394.35
246,2/27/17,1374.93


In [18]:
TEL_df = get_return(TEL_df, d=1)
TEL_df =  get_change_in_value_df(TEL_df, N=700)
TEL_df

,dt,close,previous,return,change_in_value
0,2/23/18,1488.74,1510.86,-0.014749,-15370.094698
1,2/22/18,1510.86,1513.72,-0.001891,-1970.824622
2,2/21/18,1513.72,1536.65,-0.015035,-15667.749059
3,2/20/18,1536.65,1476.37,0.040018,41703.842890
4,2/16/18,1476.37,1475.71,0.000447,465.975103
...,...,...,...,...,...
243,3/2/17,1430.29,1408.94,0.015040,15673.009003
244,3/1/17,1408.94,1394.35,0.010409,10847.710303
245,2/28/17,1394.35,1374.93,0.014026,14616.263663
246,2/27/17,1374.93,1367.68,0.005287,5509.622658


In [23]:
M = len(TEL_df["change_in_value"])-1
M

247

In [25]:
p =99
alpha = p/100
1-alpha

0.010000000000000009

In [28]:
type(np.floor((1-alpha)*M))

numpy.float64

In [36]:
k = int(np.floor((1-alpha)*M))
abs(TEL_df["change_in_value"].nsmallest(k).values[-1])

60730.66451735293

In [32]:
TEL_df["change_in_value"].sort_values()

68    -79559.535694
43    -60730.664517
10    -52200.460251
161   -50412.452299
49    -47613.180665
           ...     
185    48898.290082
198    49220.599037
239    70041.483709
19     83705.595030
247             NaN
Name: change_in_value, Length: 248, dtype: float64

## Question on N-Asset Portfolio:
Today is February 23, 2018. You are a portfolio risk manager who is assigned to analyze the market risk of a portfolio of 700 PLDT (TEL) shares, 300 Ayala Corporation (AC) shares, and 500 URC shares. What is the portfolio's one-day 99% VaR using historical approach?